## 4. Реализация приложения. Приложение может быть консольное, оконное или веб-приложение по выбору.

In [2]:
# Импортируем необходимые библиотеки

import numpy as np                                      
import pandas as pd                                    
import xlrd                                           

from sklearn.preprocessing import StandardScaler        
from sklearn.model_selection import train_test_split    
from sklearn.ensemble import RandomForestRegressor      

import tkinter as tk                                    # для создания оконного приложения
from tkinter import ttk
from tkinter import *
from tkinter import messagebox

In [3]:
dataset = pd.read_excel('ebw_data.xlsx')               

In [4]:
std_scaler = StandardScaler()                                                         
dataset_std = std_scaler.fit_transform(np.array(dataset[['IW', 'IF', 'VW', 'FP']]))   
dataset_std_df = pd.DataFrame(data = dataset_std, columns = ['IW', 'IF', 'VW', 'FP'])

In [5]:
writer = pd.ExcelWriter('output.xlsx')
dataset_std_df.to_excel(writer)
writer.save()

# пока сидела на всевозможных форумах, вычитала, что нельзя обучать модель на стандартизованных данных, 
# а потом подавать для предсказания нестандартизованные.
# я по всякому пыталась стандартизировать подаваемые в приложение данные, но у меня ничего не получалось.
# поэтому решила так: скачаю файл Excel со стандартизированными данными, найду зависимость между стандартизированными 
# и нестандартизироваными данными (там оказалась обычная линейная зависимость типа y=kx+b, и просто "стандартизирую" подаваемые данные вручную)

In [6]:
y1_columns = ['Depth']                                                 
x1_columns = ['IW', 'IF', 'VW', 'FP']                                 

y1 = dataset.loc[:, y1_columns]                                      
x1 = dataset_std_df.loc[:, x1_columns]  

x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, 
                                                        test_size=0.25, 
                                                        shuffle = True, 
                                                        random_state = 15)


y2_columns = ['Width']                                               
x2_columns = ['IW', 'IF', 'VW', 'FP']

y2 = dataset.loc[:, y2_columns]
x2 = dataset_std_df.loc[:, x2_columns]

x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, 
                                                        test_size=0.25, 
                                                        shuffle = True, 
                                                        random_state = 15)
RFG1 = RandomForestRegressor()
RFG1.fit(x1_train, np.ravel(y1_train))
y1_test_pred = RFG1.predict(x1_test)

RFG2 = RandomForestRegressor()
RFG2.fit(x2_train, np.ravel(y2_train))
y2_test_pred = RFG2.predict(x2_test)


In [7]:
def calculate_DepthWidth():
   IW = float(IW_tf.get())
   IW = 0.6*IW-27.4                             # это мы в Excel посчитали, как надо масштабировать
   IF = float(IF_tf.get())
   IF = 0.19569842*IF-27.6587103
   VW = float(VW_tf.get())
   VW = 0.48858776*VW-4.22085534
   FP = float(FP_tf.get())
   FP = 0.04685213*FP-3.6700834
   Depth = RFG1.predict([[IW, IF, VW, FP]])
   DEP = np.round(Depth, 3)
   Width = RFG2.predict([[IW, IF, VW, FP]])
   WID = np.round(Width, 3)
   return messagebox.showinfo('Ура! Всё посчиталось', f'Глубина сварного шва (Depth) равна {DEP},\
                              \n ширина сварного шва (Width) равна {WID}')

window = Tk()
window.iconbitmap('Иконкa.ico')                 # это иконка (а то там по умолчанию какое-то пёрышко, мне не понравилось)
window.title('Рассчёт прогнозируемой глубины (Depth) и ширины (Width) сварного шва')      # это заголовок
window.geometry('800x250')                      # это размер окна
  
frame = Frame(window, padx=10, pady=10)
frame.pack(expand=True)
 
IW_lb = Label(frame, text="Введите величину сварочного тока (IW)  ")
IW_lb.grid(row=3, column=1)
 
IF_lb = Label(frame, text="Введите ток фокусировки электронного пучка (IF)  ")
IF_lb.grid(row=4, column=1)

VW_lb = Label(frame, text="Введите скорость сварки (VW)   ")
VW_lb.grid(row=5, column=1)

FP_lb = Label(frame, text="Введите расстояние от поверхности образцов до электронно-оптической системы (FP)   ")
FP_lb.grid(row=6, column=1)
 
IW_tf = Entry(frame)
IW_tf.grid(row=3, column=2, pady=5)
 
IF_tf = Entry(frame)
IF_tf.grid(row=4, column=2, pady=5)

VW_tf = Entry(frame)
VW_tf.grid(row=5, column=2, pady=5)

FP_tf = Entry(frame)
FP_tf.grid(row=6, column=2, pady=5)
 
cal_btn = Button(frame,
                 text='Рассчитать прогнозируемую глубину (Depth) \nи ширину (Width) сварного шва',
                 command=calculate_DepthWidth)
cal_btn.grid(row=7, column=2)
 
window.mainloop()

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
